In [1]:
!pip3 install deepface

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
from deepface import DeepFace

In [3]:
db = pd.read_feather("./vgg_face_dataset_db.feather")

In [4]:
db.head()

,identity,VGG-Face_representation,Facenet_representation,OpenFace_representation,DeepFace_representation
0,/Users/admin/Documents/tmp/python_play/face/da...,"[0.004740223288536072, -0.0013511808356270194,...","[0.7250798940658569, 1.084446668624878, -0.116...","[0.03616740182042122, -0.11801913380622864, 0....","[0.0, 0.0, 0.0, 0.3374479413032532, 2.53953027..."
1,/Users/admin/Documents/tmp/python_play/face/da...,"[0.008823225274682045, 0.014276314526796341, -...","[-0.8137578964233398, 0.6834025382995605, 0.63...","[0.11470276117324829, -0.14653193950653076, 0....","[0.0, 0.0, 0.0, 1.0103286504745483, 5.41690826..."
2,/Users/admin/Documents/tmp/python_play/face/da...,"[0.020516492426395416, 0.009519693441689014, -...","[1.5883285999298096, 2.526050329208374, -1.013...","[0.03891799598932266, -0.06711485236883163, -0...","[0.0, 0.0, 0.0, 0.2389901876449585, 1.51561474..."
3,/Users/admin/Documents/tmp/python_play/face/da...,"[0.022623969241976738, 0.019174646586179733, -...","[-0.452881783246994, 1.232438087463379, -0.068...","[0.11516433209180832, -0.060278184711933136, -...","[0.0, 0.0, 0.0, 0.35286810994148254, 3.4454002..."
4,/Users/admin/Documents/tmp/python_play/face/da...,"[0.011912306770682335, 0.009357951581478119, -...","[-0.5669993162155151, 2.130279779434204, -0.18...","[0.06373895704746246, -0.05842230096459389, 0....","[0.0, 0.0, 0.0, 0.02194952964782715, 2.1955332..."


In [2]:
def encode_vgg_face(path):
  return DeepFace.represent(path, model_name="VGG-Face", detector_backend="mtcnn")

def encode_deepid_face(path):
  return DeepFace.represent(path, model_name="DeepID", detector_backend="mtcnn")

In [5]:
def findEuclideanDistance(source_representation, test_representation):
    if type(source_representation) == list:
        source_representation = np.array(source_representation)

    if type(test_representation) == list:
        test_representation = np.array(test_representation)

    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

thresholds = {
		'VGG-Face': {'cosine': 0.40, 'euclidean': 0.60, 'euclidean_l2': 0.86}
}

In [9]:
def compare_vgg_with_db(my_face, db):
  distances = []
  for idx, data in db["VGG-Face_representation"].iteritems():
    d = findCosineDistance(my_face, data)
    distances.append(d)
  
  return pd.DataFrame.from_dict({
    "dist": distances
  }).sort_values(by="dist", ascending=True)
    

In [10]:
def compare_img(path):
  my_face = encode_vgg_face(path)
  res = compare_vgg_with_db(my_face, db)
  res = res[res["dist"] < 0.4]
  return res

In [11]:
compare_img("/Users/admin/Downloads/IMG_3346.JPG")

,dist
310,0.209833
304,0.218824
306,0.224306
317,0.228145
1493,0.233930
...,...
2041,0.399041
2654,0.399150
3432,0.399570
3684,0.399869


In [12]:
compare_img("/Users/admin/Downloads/brother.jpg")

,dist
1489,0.217627
1492,0.223620
1485,0.243224
2427,0.252613
2428,0.256794
...,...
396,0.399817
2581,0.399940
1857,0.399966
2440,0.399973


In [13]:
compare_img("/Users/admin/Documents/tmp/python_play/face/dataset/vgg_face_dataset/faces/Junichiro_Koizumi/Junichiro_Koizumi_0059.jpg")

,dist
2524,0.021529
2484,0.071182
2527,0.079082
2526,0.084874
2497,0.089878
...,...
3285,0.399634
47,0.399702
459,0.399743
77,0.399901


In [14]:
compare_img("/Users/admin/Downloads/selfie_sleepy.jpg")

,dist
1489,0.233977
2427,0.239235
2428,0.242242
1421,0.244472
1420,0.250804
...,...
3288,0.399692
3466,0.399728
2567,0.399852
2234,0.399946


In [24]:
def update_db(db, path):
  my_face = encode_vgg_face(path)
  db = db.append({"identity": path, "VGG-Face_representation": my_face}, ignore_index=True)
  return db

In [27]:
db = update_db(db, "/Users/admin/Downloads/selfie_sleepy.jpg")

In [28]:
compare_img("/Users/admin/Downloads/IMG_3346.JPG")

,dist
3689,0.197998
3688,0.197998
310,0.209833
304,0.218824
306,0.224306
...,...
2041,0.399041
2654,0.399150
3432,0.399570
3684,0.399869


In [29]:
compare_img("/Users/admin/Downloads/selfie_sleepy.jpg")

,dist
3689,4.440892e-16
3688,4.440892e-16
1489,2.339767e-01
2427,2.392346e-01
2428,2.422423e-01
...,...
3288,3.996920e-01
3466,3.997284e-01
2567,3.998518e-01
2234,3.999464e-01


In [4]:
res = encode_deepid_face("/Users/admin/Downloads/me.png")

(1, 55, 47, 3)


In [8]:
res

[0.0,
 0.0,
 20.870649337768555,
 29.81072425842285,
 0.0,
 0.0,
 31.13530158996582,
 0.0,
 0.0,
 0.0,
 5.160918712615967,
 0.0,
 16.361160278320312,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 12.635902404785156,
 0.0,
 0.0,
 0.0,
 0.0,
 23.826326370239258,
 0.0,
 7.553755760192871,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 32.920780181884766,
 15.193655014038086,
 13.959370613098145,
 40.493289947509766,
 0.0,
 12.417348861694336,
 1.6788386106491089,
 7.50777006149292,
 35.63104248046875,
 0.0,
 19.435087203979492,
 0.0,
 0.0,
 16.784631729125977,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 27.366514205932617,
 0.0,
 0.0,
 13.678277969360352,
 0.0,
 0.0,
 25.86917495727539,
 28.302820205688477,
 12.798733711242676,
 0.0,
 36.29988098144531,
 0.0,
 0.0,
 18.52462387084961,
 10.317431449890137,
 6.647881984710693,
 0.0,
 0.0,
 0.0,
 29.709619522094727,
 10.0660400390625,
 0.5792498588562012,
 24.71436309814453,
 0.0,
 0.0,
 24.709854125976562,
 11.12615966796875,
 0.0,
 0.0,
 44.404876708984375,
 53.556